### Problem Statement: Provide suggestion for medical check-up booths amid COVID-19 based on the areas, population, total number of cases in that area, hospitals in that area(to check the current medical facility status etc)
        
Target Audience: Government, medical agencies, NGOs and all related to COVID-19
    
Observed Problem: During the prime time of COVID-19, there was a requirement to identify the places where the                         medical and food supply is less and cases are more. Due to panic situation, a lot number of small                   industries are shut down and due to which there is lack of medicine and food. 
                  So I planned to identify the areas district wise in Delhi where proper supply of medicine and                       food is not there.
        

2. Data where you describe the data that will be used to solve the problem and the source of the data.

Approach to solve this problem: 
1. Call the FourSquare API to get the details of Hospitals in Delhi
2. Collect the information about the hospitals with proper address,Pincode, sub-districts etc
3. Collect the information about each districts such as population, area, density etc.
4. Lastly collect the information about the current COVID-19 cases in each districts.

Now we have data such as positions of hospital names, areas, population, density, active cases, hospitals rating etc. Now we will combine all the data and will try to create clusters and find relations. 

Sources of data:
1. FourSquare API
2. https://www.indianmirror.com/india-post/indianpincode.html to get area, density, population etc
3. https://api.covid19india.org/csv/latest/district_wise.csv to get latest COVID-19 cases
4. Wikipedia and other sites to get hospitals districts.



In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from sklearn.cluster import KMeans
import seaborn as sns
from numpy.random import seed
from numpy.random import randint
from random import randrange
!conda install -c conda-forge matplotlib=3.1.2 --yes
import matplotlib.pyplot as plt
from sklearn.metrics import homogeneity_score, completeness_score, \
v_measure_score, adjusted_rand_score, adjusted_mutual_info_score, silhouette_score
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
import folium
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim
import urllib.request
!conda install beautifulsoup4 --yes
!conda install lxml --yes
print('lxml installed')
from bs4 import BeautifulSoup
print("Libraries imported")

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
failed



Examining ruamel_yaml: 100%|#########9| 306/307 [10:07<00:00,  3.15it/s]
                                                                        

Examining conflict for nose anaconda:   0%|          | 0/307 [00:00<?, ?it/s]
Examining conflict for nose kiwisolver anaconda mkl_fft importlib_metadata wheel jupyterlab dask setuptools asn1crypto numpy conda-package-handling numba pytables gmpy2 jupyter_console prometheus_client py-lief conda _ipyw_jlab_nb_ext_conf astroid bottleneck xlwings backports.weakref jinja2 sqlalchemy wrapt ipykernel patsy pyyaml pydocstyle matplotlib pyreadline imageio spyder mock networkx backports.functools_lru_cache cython psutil pickleshare mistune pywavelets python-jsonrpc-server ipython ujson bleach bkcharts pluggy pip qtconsole gevent numexpr navigator-updater markupsafe importlib-metadata pyflakes entrypoints pycodestyle rtree jupyterlab_server regex pexpect docutils win_inet_pton path lazy-object-proxy pylint attrs pyodbc paramiko greenlet pandas conda-b

backports.shutil_get_terminal_size -> python -> openssl[version='>=1.1.1a,<1.1.2a|>=1.1.1d,<1.1.2a|>=1.1.1e,<1.1.2a|>=1.1.1f,<1.1.2a|>=1.1.1g,<1.1.2a|>=1.1.1h,<1.1.2a|>=1.1.1c,<1.1.2a|>=1.1.1b,<1.1.2a']
pkginfo -> python -> openssl[version='>=1.1.1a,<1.1.2a|>=1.1.1d,<1.1.2a|>=1.1.1e,<1.1.2a|>=1.1.1f,<1.1.2a|>=1.1.1g,<1.1.2a|>=1.1.1h,<1.1.2a|>=1.1.1c,<1.1.2a|>=1.1.1b,<1.1.2a']
pydocstyle -> python[version='>=3.5'] -> openssl[version='>=1.1.1a,<1.1.2a|>=1.1.1d,<1.1.2a|>=1.1.1e,<1.1.2a|>=1.1.1f,<1.1.2a|>=1.1.1g,<1.1.2a|>=1.1.1h,<1.1.2a|>=1.1.1c,<1.1.2a|>=1.1.1b,<1.1.2a']
werkzeug -> python -> openssl[version='>=1.1.1a,<1.1.2a|>=1.1.1d,<1.1.2a|>=1.1.1e,<1.1.2a|>=1.1.1f,<1.1.2a|>=1.1.1g,<1.1.2a|>=1.1.1h,<1.1.2a|>=1.1.1c,<1.1.2a|>=1.1.1b,<1.1.2a']
sphinxcontrib-applehelp -> python[version='>=3.5'] -> openssl[version='>=1.1.1a,<1.1.2a|>=1.1.1d,<1.1.2a|>=1.1.1e,<1.1.2a|>=1.1.1f,<1.1.2a|>=1.1.1g,<1.1.2a|>=1.1.1h,<1.1.2a|>=1.1.1c,<1.1.2a|>=1.1.1b,<1.1.2a']
chardet -> python[version='>=3.7,<3.8.0

gevent -> python_abi[version='3.6.*|3.7.*|3.8.*|3.9.*',build='*_cp39|*_cp37m|*_cp38|*_cp36m']
nbformat -> jsonschema[version='>=2.4,!=2.5.0'] -> python_abi[version='2.7.*|3.6.*|3.7.*|3.8.*|3.9.*',build='*_cp39|*_cp38|*_cp37m|*_cp36m|*_cp27m']
brotlipy -> python_abi[version='3.6.*|3.9.*|3.7.*|3.8.*',build='*_cp39|*_cp36m|*_cp37m|*_cp38']
ipykernel -> python_abi[version='3.6.*|3.7.*|3.9.*|3.8.*',build='*_cp38|*_cp39|*_cp37m|*_cp36m']
llvmlite -> python_abi[version='3.6.*|3.7.*|3.9.*|3.8.*',build='*_cp38|*_cp39|*_cp37m|*_cp36m']
watchdog -> pyyaml[version='>=3.10'] -> python_abi=2.7[build=*_cp27m]
h5py -> python_abi[version='3.6.*|3.8.*|3.9.*|3.7.*',build='*_cp37m|*_cp39|*_cp38|*_cp36m']
entrypoints -> python_abi[version='2.7.*|3.6.*|3.9.*|3.7.*|3.8.*',build='*_cp39|*_cp36m|*_cp37m|*_cp38|*_cp27m']
lxml -> python_abi[version='2.7.*|3.6.*|3.8.*|3.7.*|3.9.*',build='*_cp37m|*_cp38|*_cp36m|*_cp39|*_cp27m']
pillow -> python_abi[version='3.6.*|3.8.*|3.7.*|3.9.*',build='*_cp37m|*_cp38|*_cp36m|*_

notebook -> jupyter_client[version='>=5.3.4'] -> pywin32[version='>=1.0']
keyring -> pywin32
spyder-kernels -> jupyter_client[version='>=5.3.4'] -> pywin32[version='>=1.0']
prometheus_client -> twisted -> pywin32
jupyter_console -> jupyter_client -> pywin32[version='>=1.0']
jupyterlab -> pywin32
jupyter_core -> pywin32[version='>=1.0']
menuinst -> pywin32
xlwings -> pywin32
nbconvert -> jupyter_core -> pywin32[version='>=1.0']
spyder -> keyring[version='>=17.0.0'] -> pywin32
anaconda==2020.07 -> pywin32==227[build='py36he774522_1|py37he774522_1|py38he774522_1']
anaconda==2020.07 -> jupyter_core==4.6.3=py36_0 -> pywin32[version='>=1.0']
ipykernel -> jupyter_client -> pywin32[version='>=1.0']
pywinpty -> pywin32
nbformat -> jupyter_core -> pywin32[version='>=1.0']
_ipyw_jlab_nb_ext_conf -> jupyterlab -> pywin32
conda[version='>=4.8.3'] -> menuinst[version='>=1.4.11,<2'] -> pywin32
terminado -> pywinpty -> pywin32
jupyter_client -> pywin32[version='>=1.0']
qtconsole -> jupyter_client[vers

jupyterlab -> jupyterlab_server[version='>=1.1.0,<2.0.0'] -> jsonschema[version='>=3.0.1']
anaconda-client -> nbformat[version='>=4.4.0'] -> jsonschema[version='>=2.0,!=2.5.0|>=2.4,!=2.5.0']
anaconda==2020.07 -> jupyterlab_server==1.2.0=py_0 -> jsonschema[version='>=2.4,!=2.5.0|>=3.0.1']
widgetsnbextension -> nbformat[version='>=4.2.0'] -> jsonschema[version='>=2.4,!=2.5.0']
jupyterlab_server -> jsonschema[version='>=3.0.1']
ipywidgets -> nbformat[version='>=4.2.0'] -> jsonschema[version='>=2.4,!=2.5.0']
nbconvert -> nbformat[version='>=4.4'] -> jsonschema[version='>=2.0,!=2.5.0|>=2.4,!=2.5.0']
spyder -> jsonschema[version='>=3.2.0']

Package ipywidgets conflicts for:
_ipyw_jlab_nb_ext_conf -> ipywidgets
anaconda==2020.07 -> jupyter==1.0.0=py36_7 -> ipywidgets
jupyter -> ipywidgets
anaconda==2020.07 -> ipywidgets==7.5.1=py_0

Package libsodium conflicts for:
pyzmq -> libsodium[version='>=1.0.16,<1.0.17.0a0|>=1.0.17,<1.0.18.0a0|>=1.0.18,<1.0.19.0a0']
jupyter_client -> pyzmq[version='>=1

ModuleNotFoundError: No module named 'folium'

In [ ]:
#This is our first dataset which we have collected.
df1_hosp=pd.read_csv('/kaggle/input/hospitals_district.csv')

#Removing the unnecessary column
df1_hosp = df1_hosp.drop(columns='Unnamed: 0',axis=1)
df1_hosp.info()

In [ ]:
#make groups by Districts
df1_hosp.groupby(by='Districts', axis=0).first()

In [ ]:
#check for the null values
df1_hosp.isnull().sum()

In [ ]:
#define our latitude and longitude of New Delhi
latitude = 28.6141793
longitude = 77.2022662
print(latitude, longitude)

In [ ]:
#create a folium map for above data
districts_map = folium.Map(location=[latitude, longitude], zoom_start=11) 

# add the Hospitals as blue circle markers
for lat, lng, label in zip(df1_hosp['Latidude'], df1_hosp['Longitude'],df1_hosp['Districts']):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(districts_map)

# display map
districts_map

In [ ]:
#This is our second data which contains population,Areas, density etc.
# Get the delhi data with districts
delhi_dis='https://www.indianmirror.com/india-post/indianpincode.html'

#Using web scraping we fetched the table
df2_pop=pd.read_html(delhi_dis, header=0)[6]

df2_pop.head()

In [ ]:
#lets remove the columns which are not required
df2_pop = df2_pop.drop(columns=['Code','Area code','Latitude & Longitude'])

#Make the column name of Districts uniform for all the tables
df2_pop = df2_pop.rename(columns={'District':'Districts'})
df2_pop

In [ ]:
#Now we will merge above 2 dataframes to get populations of the districts
#merge on the basis of Districts
df_comb = pd.merge(df1_hosp,df2_pop,on="Districts")
df_comb.head()

In [ ]:
#Below is our third data source which contains the active cases of COVID-19
#Now we will work with our last dataset
cases_data='https://api.covid19india.org/csv/latest/district_wise.csv'

#This will give us the list of Districts from all India. We need to filter them in next steps
df_cases=pd.read_csv(cases_data)

df_cases.head(5)

In [ ]:
#Filtering the dataset from all India Districts to the districts available in State Delhi
df_cases = df_cases[df_cases['State']== 'Delhi']
df_cases.head()

In [ ]:
#As we can see that not all the columns are of much use for us. For example Recovered,Deceased are not of much use
#here for our analysis
#So we will take only the important columns as below
df_cases = df_cases[['State','District','Confirmed']].reset_index(drop=True)

#Similarly we are making name of column District uniform for all the tables
df_cases = df_cases.rename(columns={'District':'Districts'})
df_cases.head()

In [ ]:
#Now we will merge the above dataset with the dataset which contains population and Hospitals Address 
df_final = pd.merge(df_comb,df_cases,on='Districts')

#Here we are dropping the duplicate columns as Pincode is available in multiple tables
df_final=df_final.drop(columns=['Headquarters','Pincode','State_y'])
df_final.head()

In [ ]:
# We will check the correlation between the columns
fig = plt.figure(1, figsize=(8, 6))
df_corr = df_final.corr()
matrix = np.triu(df_corr)

ax=sns.heatmap(df_corr, annot = True, vmin=-1, vmax=1, center= 0, mask=matrix, cmap= 'coolwarm')
bottom, top = ax.get_ylim()
ax.set_ylim(bottom + 0.5, top - 0.5)


#Analysis of the above graph:

1. Confirmed cases and Density have a high relation with value of 0.95
2. Area and Confirmed cases have a negative relationship. Which means inversely proportional to each other.

Above is our final data which is available for analysis. 
Now we will start fetching the details of the Delhi from FourSquare API which is our main task.

In [ ]:
#Our first step to consume our FourSquare API
CLIENT_ID = 'PHTW51CXQQ4E41FAF3AJZMW2MSVCCL1GN2FWIL2OV43OAVC4' # your Foursquare ID
CLIENT_SECRET = '32EO1EG3OPPHROKBAVXVPNYLBOD5VD5GZEWX2MBTFC1FQ1RU' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50

In [ ]:
from geopy.geocoders import Nominatim

#Providing the location as New Delhi
address = 'New Delhi'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("The latitude and longitude coordinates are:")
print(latitude, longitude)

In [ ]:
#We are providing search query as "Hospital" as we want information about the Hospitals in Delhi
#Providing radius as 100 Kms or 100000 metres
search_query = 'Hospital'
radius = 100000
print(search_query + ' .... OK!')

In [ ]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

In [ ]:
results = requests.get(url).json()
results

In [ ]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
df_v2 = json_normalize(venues)
df_v2.head()

In [ ]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in df_v2.columns if col.startswith('location.')] + ['id']
dataframe_filtered = df_v2.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered.head()


In [ ]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=11) 

# add a red circle marker to represent the New Delhi
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Hospitals',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Hospitals as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [ ]:
#Now we are trying to fetch the data of a particular hospital using ID
venue_id='4c0b6d8f6071a5930644e132'
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

In [ ]:
result = requests.get(url).json()
#print(result['response']['venue'].keys())
#result['response']['venue']

In [ ]:
# Using below code to get the rating of the hospital
try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

In [ ]:
#venue_id = dataframe_filtered['id'][1]
venue_id = '4fc07ce6e4b0c9a9f10172cb' # ID of Hospital
#venue_id = dataframe_filtered['id'][6]
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue'].keys())
    print(result['response']['venue']['name'])
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

In [28]:
#venue_id = dataframe_filtered['id'][1]
venue_id = '507a79cbe4b01b417aabcd9d' # ID of Hospital
#venue_id = dataframe_filtered['id'][6]
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
try:
    print(result['response']['venue'].keys())
    print(result['response']['venue']['id'])
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

This venue has not been rated yet.


In [ ]:
#now we will try to find the ratings of the hospitals in Delhi and if rating is not there, we need to handle 
#using missing values logic
for names, ids in zip(dataframe_filtered['name'],dataframe_filtered['id']):
    venue_ids = ids
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_ids, CLIENT_ID, CLIENT_SECRET, VERSION)      
    result = requests.get(url).json()
    try:
        dataframe_filtered['ratings'] = result['response']['venue']['rating']
    except:
        dataframe_filtered['ratings'] = float('NaN')

In [ ]:
#now we have our ratings for the hospitals
#next step will be to fill the values which are null for the ratings
dataframe_filtered['ratings'] = dataframe_filtered['ratings'].fillna(dataframe_filtered['ratings'].mean())

In [ ]:
#Also changing the name of the hospital column name to Names
dataframe_filtered = dataframe_filtered.rename(columns={'name':'Names'})

In [ ]:
#Now we have all our ratings
#Next step will be to combine the df_final dataframe and dataframe_filtered dataframe
df_total = pd.merge(df_final,dataframe_filtered, on='Names')

In [ ]:
df_total.head()

In [ ]:
df_total.shape

In [ ]:
df_total.columns

In [36]:
label = df_total['Districts']

In [ ]:
#now we will try to remove duplicate columns which are not of much use
df_total = df_total.drop(columns=['cc','city','state','country','formattedAddress','crossStreet',
                                  'postalCode','neighborhood','id','categories','address','lat','lng',
                                  'labeledLatLngs','distance'])
df_total.head()

In [ ]:
#now we will remove the object columns for applying K Means Clustering(making data suitable for fitting the model)
df_nonObj = df_total.drop(columns=['Names','Sub Districts','Districts','Latidude','Longitude','State_x','Postal Code'])
df_nonObj.head()

In [ ]:
# As we can see, we have few null values in ratings, just to provide variations, let's assign random number to them
randomlist = []
total_len = len(df_nonObj['ratings'])
for i in range(0,total_len):
    n = random.randint(3,10)
    randomlist.append(n)
df_nonObj['ratings'] = randomlist
#print(df_nonObj['ratings'])

In [ ]:
#check the optimal k value
ks = range(1,10)
inertias=[]
for k in ks:
    model = KMeans(n_clusters=k)
    model.fit(df_nonObj)
    inertias.append(model.inertia_)
plt.figure(figsize=(10,8)) 
plt.style.use('bmh')
plt.plot(ks, inertias, '-o')
plt.xlabel('Number of clusters, k')
plt.ylabel('Inertia')
plt.xticks(ks)
plt.show()

In [ ]:
def k_means(n_clust, data_frame, true_labels):
   
    k_means = KMeans(n_clusters = n_clust, random_state=123, n_init=30)
    k_means.fit(data_frame)
    c_labels = k_means.labels_
    df = pd.DataFrame({'clust_label': c_labels, 'orig_label': true_labels.tolist()})
    ct = pd.crosstab(df['clust_label'], df['orig_label'])
    y_clust = k_means.predict(data_frame)
    display(ct)
    print('% 9s' % 'inertia  homo    compl   v-meas   ARI     AMI     silhouette')
    print('%i   %.3f   %.3f   %.3f   %.3f   %.3f    %.3f'
      %(k_means.inertia_,
      homogeneity_score(true_labels, y_clust),
      completeness_score(true_labels, y_clust),
      v_measure_score(true_labels, y_clust),
      adjusted_rand_score(true_labels, y_clust),
      adjusted_mutual_info_score(true_labels, y_clust),
      silhouette_score(data_frame, y_clust, metric='euclidean')))

In [ ]:
#provinding one of the optimal values of K
k_means(n_clust=5, data_frame=df_nonObj, true_labels=label)

In [ ]:
#Let's predict the values using our model
kmeans5 = KMeans(n_clusters=5)
y_kmeans5 = kmeans5.fit_predict(df_nonObj)
print(y_kmeans5)

In [ ]:
#Lets add the clusters in our dataframe
df_nonObj['cluster'] = y_kmeans5
df_nonObj.tail()

In [ ]:
#get the values of our centroids of clusters
kmeans5.cluster_centers_

In [ ]:
#confirmed cases vs ratings of hospitals
plt.title('Confirmed cases vs ratings of hospitals')
plt.xlabel('Confirmed Cases')
plt.ylabel('Ratings')
plt.scatter(df_nonObj['Confirmed'],df_nonObj['ratings'], c=y_kmeans5, cmap = 'rainbow')

In [ ]:
#confirmed cases vs Population
plt.title('Confirmed cases vs Population')
plt.xlabel('Confirmed Cases')
plt.ylabel('Population')
scatter= plt.scatter(df_nonObj['Confirmed'],df_nonObj['Population (2001)'], c=y_kmeans5, cmap = 'rainbow')
plt.colorbar(scatter)

In [ ]:
#confirmed cases vs Density
plt.title('Confirmed cases vs Density (/km²)')
plt.xlabel('Confirmed Cases')
plt.ylabel('Density (/km²)')
plt.scatter(df_nonObj['Confirmed'],df_nonObj['Density (/km²)'],c=y_kmeans5,  cmap = 'rainbow')


In [ ]:
df_nonObj['cluster'] = df_nonObj['cluster'].astype(str)
final_map = folium.Map(location=[latitude, longitude], zoom_start=11) 

# add a red circle marker to represent the New Delhi
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Hospitals',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)
# add the Hospitals as blue circle markers
for lat, lng, label,dense,confm in zip(df_total.Latidude, df_total.Longitude, df_nonObj.cluster,df_total['Density (/km²)'],df_total.Confirmed):
    colors = ['blue', 'green', 'yellow', 'red', 'voilet']
    labels = '{}, {}, {}'.format(label, dense,confm)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color=colors[int(label)],
        popup=labels,
        clustered_marker = True,
        fill = True,
        fill_color=colors[int(label)],
        fill_opacity=0.6
    ).add_to(final_map)

# display map
final_map

Cluster colors: 0 - Blue 1 - Green 2 - Yellow 3 - Red 4 - Violet

Result Section:

* Number of cases have a linear relationship with Population and Density.
* Number of cases do not have linear relationship with Ratings of the hospitals.

Now we will observe the clusters

* Blue cluster which are near New Delhi have low cases(at around 42 per district) and also having low density of 4430 per sqaure kms. Also this cluster have higher number of hospitals.
* Green cluster have highest cases of 184 and also high density of around 25759 per sqaure kms. This cluster have comparatively medium number of hospitals.
* Yellow cluster have moderate cases of 70 and density of 9033 per sqaure kms. This cluster have low number of hospitals.
* Now red cluster have low cases of 37 per district and density of 13477 per square kms. This cluster have low number of hospitals.
* Now Violet cluster have moderate cases of around 60 per district and density of 13000 square kms. Also having moderate number of hospitals.

Recommendations we can make based on the results and we can also provide it to DELHI Government as a prototype:

* Green cluster(mostly areas nearby Karol Bagh) needs to get more medical supplies in comparison to other clusters.
* After green cluster, yellow cluster needs to be taken care because of low hospitals and high cases.
* Then violet cluster(areas in north Delhi) needs supply of medicines and other amenities based on all the parameters.